In [15]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.dataset as dsF
import pyarrow.compute as pc
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from multiprocessing import Pool
import json
from source import *
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

import scienceplots

plt.style.use(['science','ieee', 'grid'])

In [19]:
import numpy as np
from typing import Union, Callable

def normalization(values: Union[np.ndarray, Callable],
                  a: Union[list, int],
                  b: Union[list, int],
                  func: bool = False,
                  n_points: int = 500):
    from numpy import trapz
    
    # --- Одномерный случай ---
    if isinstance(a, int) and isinstance(b, int):
        if func:
            x = np.linspace(a, b, n_points)
            y = values(x)  # values — функция
        else:
            x = np.linspace(a, b, len(values))
            y = values
        return trapz(y, x=x)
    
    # --- Многомерный случай ---
    if isinstance(a, list) and isinstance(b, list):
        if func:
            grids = [np.linspace(a[i], b[i], n_points) for i in range(len(a))]
            mesh = np.meshgrid(*grids, indexing='ij')
            vals = values(*mesh)
        else:
            vals = values
        
        summ = vals
        for dim in range(len(a)):
            x_dim = np.linspace(a[dim], b[dim], summ.shape[0])
            summ = trapz(summ, x=x_dim, axis=0)
        return summ
    
    raise ValueError("Invalid argument types")


In [27]:
import numpy as np
from typing import Union, Callable, List

def normalization(values: Union[np.ndarray, Callable],
                  a: Union[float, List[float]],
                  b: Union[float, List[float]],
                  func: bool = False,
                  n_points: int = 200):
    from numpy import trapz

    # Приводим к списку, даже если это одно число
    if isinstance(a, (int, float)) and isinstance(b, (int, float)):
        a = [a]
        b = [b]

    dims = len(a)

    # Если передана функция — строим сетку
    if func:
        grids = [np.linspace(a[i], b[i], n_points) for i in range(dims)]
        mesh = np.meshgrid(*grids, indexing='ij')
        coords = np.stack([m.ravel() for m in mesh], axis=-1)  # (N, dims)
        vals = values(coords).reshape([n_points]*dims)
    else:
        vals = values

    summ = vals
    for dim in range(dims):
        x_dim = np.linspace(a[dim], b[dim], summ.shape[0])
        summ = trapz(summ, x=x_dim, axis=0)
    return summ


In [29]:
# f(x, y) = x + y на [0,1] × [0,2], интеграл = 3
def f(coords):
    x = coords[:, 0]
    y = coords[:, 1]
    return x + y

print(normalization(f, [0, 0], [1, 2], func=True, n_points=300))  # ≈ 3.0

# f(x) = x^2 на [0,2], интеграл = 8/3
def f1(coords):
    x = coords[:, 0]
    return x**2

print(normalization(f1, [0], [2], func=True, n_points=500))  # ≈ 2.6667


2.9999999999999996
2.6666720213975044


C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1369914790.py:30: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  summ = trapz(summ, x=x_dim, axis=0)


In [ ]:
wbin_x, a_x, b_x = 0.1, 0, 1.2
wbin_y, a_y, b_y = 0.01, 5.25, 5.5
bins_x = np.linspace(a_x, b_x, int((b_x - a_x) // wbin_x)+2)
bins_y = np.linspace(a_y, b_y, int((b_y - a_y) // wbin_y)+2)

A_list = []
A_errors = []
plt_s = (12/1.7, 6/1.7)
pull_s = (12/6, 6/6)

n_toys = 100
fit_results = []

for i in range(n_toys):
    toy_sample = gen_toy_nd(
        toy_pdf,
        bounds=[[0, 1.2], [5.25, 5.5]],
        size=int((N1+N2)/6),
        grid_points=100
    )
    rez, norm, errors = max_lik_ext(
        f_fit,
        toy_sample.T,
        {"w0": 0.3, "w1": 0.5},
        a = [0.001, 5.25],
        b = [1.2, 5.5],
        err_need=True
    )

    # Пересчёт в физические A0, A1, A2
    A0 = rez["w0"]
    A1 = (1 - rez["w0"]) * rez["w1"]
    A2 = 1 - A0 - A1
    
    fit_results.append(rez)
    A_list.append([A0, A1])
    # Ошибки пересчитываются через производные (пока оставим как для w0, w1)
    A_errors.append([errors.get("w0", np.nan), errors.get("w1", np.nan)])

    # --- рисование только первых 2 игрушек ---
    if i >= 2:
        continue

    fig, axs = plt.subplots(1, 2, figsize=(2*plt_s[0], plt_s[1]*1.25))
    x_centers = 0.5 * (bins_x[:-1] + bins_x[1:])
    y_centers = 0.5 * (bins_y[:-1] + bins_y[1:])
    X, Y = np.meshgrid(x_centers, y_centers, indexing='ij')

    counts, _ = np.histogram(toy_sample.T[0], bins=bins_x)
    sources = f_fit([X, Y], **rez, stak=True)
    fit_proj_x = [np.trapz(src, y_centers, axis=1) * wbin_x * norm for src in sources]

    N_signal = A0 * norm
    N_bsb = A1 * norm
    N_ubs = A2 * norm

    labels = [
        rf"$\mathrm{{Signal}}\ (N={N_signal:.0f})$",
        rf"$B_s \to D(\ell \nu)\ell \nu\ (N={N_bsb:.0f})$",
        rf"$\mathrm{{FEI miss ID}}\ (N={N_ubs:.0f})$"
    ]

    ax = axs[0]
    ax.errorbar(x_centers, counts, yerr=np.sqrt(counts), fmt='o', label="Toy Data")
    ax.hist([x_centers]*len(fit_proj_x), bins=bins_x, weights=fit_proj_x,
            stacked=True, alpha=0.55, edgecolor="black", linewidth=1.5, label=labels)
    ax.set_ylabel(fr'$\mathrm{{Events}}\,/\,{wbin_x}\,\mathrm{{GeV}}$')
    ax.set_xlabel(r'$E_{\gamma}^\mathrm{ROE}\ \mathrm{GeV}$')
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), framealpha=0.85)

    counts, _ = np.histogram(toy_sample.T[1], bins=bins_y)
    fit_proj_y = [np.trapz(src, x_centers, axis=0) * wbin_y * np.sum(counts) for src in sources]

    ax = axs[1]
    ax.errorbar(y_centers, counts, yerr=np.sqrt(counts), fmt='o', label="Toy Data")
    ax.hist([y_centers]*len(fit_proj_y), bins=bins_y, weights=fit_proj_y,
            stacked=True, alpha=0.55, edgecolor="black", linewidth=1.5, label=labels)
    ax.set_ylabel(fr'$\mathrm{{Events}}\,/\,{wbin_y}\,\mathrm{{GeV}}$')
    ax.set_xlabel(r'$M_{B_s^0}\ \mathrm{GeV}$')
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), framealpha=0.85)

    plt.tight_layout()
    plt.savefig(f"output/toy_mc_fit_{i}.pdf")
    plt.show()

In [ ]:
# --- анализ pull ---
A_array = np.array(A_list)
A_errors = np.array(A_errors)
bins = np.linspace(-5, 5, 16)

A0_true = 0
A1_true = N2/(N1+N2)
A2_true = N1/(N1+N2)

delta_signal = (A_array[:, 0] - A0_true) / A_errors[:, 0]
delta_bsb = (A_array[:, 1] - A1_true) / A_errors[:, 1]
delta_ubs = ((1 - A_array[:, 0] - A_array[:, 1]) - A2_true) / (A_errors[:, 0] + A_errors[:, 1])

counts_signal, bin_edges = np.histogram(delta_signal, bins=bins, density=True)
counts_bsb, _ = np.histogram(delta_bsb, bins=bins, density=True)
counts_ubs, _ = np.histogram(delta_ubs, bins=bins, density=True)

bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

plt.bar(bin_centers, counts_signal, width=np.diff(bins), alpha=0.65, label=r"$\mathrm{Imp \ signal}$", align='center')
plt.bar(bin_centers, counts_bsb, width=np.diff(bins), label=r"$\mathrm{Imp \ B_s \to D(\ell \nu)\ell \nu}$",
        align='center', facecolor='none', edgecolor=colors[1], hatch='//', linewidth=3)
plt.bar(bin_centers, counts_ubs, width=np.diff(bins), label=r"$\mathrm{Imp \ FEI \ miss \ ID}$",
        align='center', facecolor='none', edgecolor=colors[2], hatch='\\\\', linewidth=3)

plt.xlabel(r"$\mathrm{Pull}$")
plt.ylabel(r"$\mathrm{Entries}$")
plt.legend()
plt.tight_layout()
plt.savefig(f"output/toy_err.pdf")
plt.show()

In [30]:
# --- Проверки ---
# Пример 1: интеграл от константы 1 на [0,1] = 1
x = np.linspace(0, 1, 100)
y = np.ones_like(x)
print("Expected: 1.0, np.trapz:", np.trapz(y, x))
print("Function:", normalization(y, 0, 1))

# Пример 2: интеграл от x на [0,1] = 0.5
x = np.linspace(0, 1, 100)
y = x
print("Expected: 0.5, np.trapz:", np.trapz(y, x))
print("Function:", normalization(y, 0, 1))

# Пример 3: интеграл от x^2 на [0,2] = 8/3 ≈ 2.666...
x = np.linspace(0, 2, 200)
y = x**2
print("Expected: ~2.6667, np.trapz:", np.trapz(y, x))
print("Function:", normalization(y, 0, 2))

Expected: 1.0, np.trapz: 1.0
Function: 1.0
Expected: 0.5, np.trapz: 0.5
Function: 0.5
Expected: ~2.6667, np.trapz: 2.6667003358501047
Function: 2.6667003358501047


C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1351524809.py:5: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  print("Expected: 1.0, np.trapz:", np.trapz(y, x))
C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1369914790.py:30: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  summ = trapz(summ, x=x_dim, axis=0)
C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1351524809.py:11: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  print("Expected: 0.5, np.trapz:", np.trapz(y, x))
C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1351524809.py:17: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  print("Expected: ~2.6667, np.trapz:", np.trapz(y,

In [22]:
x = np.linspace(0, 1, 50)
y = np.linspace(0, 2, 80)
X, Y = np.meshgrid(x, y, indexing='ij')
values = X + Y  # f(x,y) = x + y

result = normalization(values, [0, 0], [1, 2])
print("Численный интеграл:", result)
print("Ожидаемое значение:", 3.0)

Численный интеграл: 3.0000000000000004
Ожидаемое значение: 3.0


C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1476186443.py:33: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  summ = trapz(summ, x=x_dim, axis=0)


In [23]:
# 1D массив
x = np.linspace(0, 1, 100)
y = np.ones_like(x)
print(normalization(y, 0, 1))  # ≈ 1

# 1D функция
print(normalization(lambda t: t**2, 0, 2, func=True))  # ≈ 8/3

# 2D функция f(x, y) = x + y
res = normalization(lambda X, Y: X + Y, [0, 0], [1, 2], func=True)
print(res)  # ≈ 3


1.0
2.6666720213975044
3.0


C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1476186443.py:19: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return trapz(y, x=x)
C:\Users\marc\AppData\Local\Temp\ipykernel_7592\1476186443.py:33: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  summ = trapz(summ, x=x_dim, axis=0)
